# Predicting Languages with READMEs

Rajaram Gautam   
Mason Sherbondy   
Sophia Stewart   
Joshua Wheeler

February 11, 2022

---
## Executive Summary
The goal of this project is to determine what language is primarily used in a repository based only on the contents of that repository's README file.

---
## Data Wrangling

We decided to use repositories that came up in a GitHub search for the word 'zombie'. We acquired the data using Zach Gulde's functions for acquiring data using GitHub's API as well as some functions defined by our group to acquire the names of the repositories. To prepare our data, we removed those repositories which had non-English characters. We then normalized, tokenized, stemmed, lemmatized, and removed stopwords from each README. Before moving on to our data exploration, we split our dataset into train, validate, and test samples.

In [ ]:
from acquire import 
from prepare import
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

## Exploratory Data Analysis

### Question 1: What is the distribution of our Inverse Document Frequencies for the most common words?

In [ ]:
# insert frequency visualizations from Josh's notebook

### Answer 1: The majority of our top words do not have a distribution that is helpful, as most of these values are 0.

### Question 2: Does the length of the README vary by programming language?

#### $H_{0}$: Average message length for languages is about the same.
#### $H_{a}$: Average message length between languages is significantly different.

### Answer 2: 

### Question 3: Do different programming languages use a different number of unique words?

Javascript: AWS, stack, response, topic, gateway, endpoint  
HTML: integration, workshop, **h1, align,** center, homepage, firefox, implementation  
C#: learning, watching, factor, pickup, animation, fire (maybe used more for modding)  
PHP: addition, kevin, bundle (not too many in training set)  
C: victim, variant, linux, letter, cpu(also used a lot in ruby), memory, block, **sudo** (potentially also more used for video game mods)
SourcePawn: plugins, csgo(name of game), bos, force, sourcemod  
C++: online (not many repos)  
Java: door, plant, sun, storage, private, endpoint, **maven** (likely used for modding)
Python: given, shotgun, exists, specified, dictionary, codeblock, allowed  
Lua: plague, gamemode, language, infection (game mods)  
Ruby: rail, rake, generated, report, csv(?), paragraph, parse

### Answer 3: Yes, different programming languages use a different number of unique words.

### Question 4:

### Answer 4: 


### Data Exploration Takeaways:



## Modeling

For our baseline prediction, we decided to use JavaScript. We made this decision because JavaScript appears most frequently in our dataset, along with C#.

For all of our models, the features we included were TF-IDF, word count, and character count.

### Baseline
- Always predict JavaScript
- 12% Accuracy on Train
- 13% Accuracy on Validate

### Model 1: Logistic Regression
- Default hyperparameters
- 23% Accuracy on Train
- 20.45% Accuracy on Validate

### Model 2: Random Forest
- max_depth = 5
- 28% Accuracy on Train
- 13.64% Accuracy on Validate

### Model 3: Decision Tree
- max_depth = 5
- 31% Accuracy on Train
- 9% Accuracy on Validate

### Takeaways:
- Our Decision Tree model was clearly overfit as it performed better than any other model on train but it had extremely poor performance on validate.
- Both of our other models also performed better on train than validate but our Logistic Regression model seemed to be the least overfit as there was less of a difference in accuracy between train and validate.
- The Logistic Regression model performed best on the validate sample
- **Our best model is the Logistic Regression model**

### Evaluating the Best Model on Test
- 13.51% Accuracy on Test
- Baseline 10.8% Accuracy on Test

---
## Conclusion

The combination of machine learning and zombie-related repositories did not provide for very useful predictions.

### Next Steps
With more time we would like to train our models using more repositories with a greater variety of subjects.